In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_file='./train.csv'
eval_file='./eval.csv'

In [3]:
train_df=pd.read_csv(train_file)
eval_df=pd.read_csv(eval_file)

In [4]:
y_train=train_df.pop('survived')
y_eval=eval_df.pop('survived')

In [5]:
categorical_columns=['sex','n_siblings_spouses','parch','class',
                     'deck','embark_town','alone']
numeric_columns=['age','fare']
feature_columns=[]

In [6]:
for categorical_column in categorical_columns:
    vocab=train_df[categorical_column].unique()
    vocab_list=tf.feature_column.categorical_column_with_vocabulary_list(
    categorical_column,vocab)
    vocab_list=tf.feature_column.indicator_column(vocab_list)
    feature_columns.append(vocab_list)

In [7]:
for categorical_column in numeric_columns:
    num_list=tf.feature_column.numeric_column(categorical_column,dtype=tf.float64)
    feature_columns.append(num_list)

In [8]:
def make_dataset(train_df,label_df,epochs=10,shuffle=True,batch_size=32):
    dataset=tf.data.Dataset.from_tensor_slices((dict(train_df),label_df))
    if shuffle:
        dataset=dataset.shuffle(10000)
    dataset=dataset.repeat(epochs).batch(batch_size)
    return dataset

In [9]:
train_dataset=make_dataset(train_df,y_train,epochs=100)
eval_dataset=make_dataset(eval_df,y_eval,shuffle=False,epochs=1)

In [10]:
model=keras.models.Sequential()
model.add(keras.layers.DenseFeatures(feature_columns))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(2,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=keras.optimizers.SGD(lr=0.01),
             metrics=['accuracy'])

In [12]:
model.fit(
    train_dataset,
    validation_data=eval_dataset,
    steps_per_epoch=18,
    validation_steps=8,
    epochs=100)

Epoch 1/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5402 - accuracy: 0.7465 - val_loss: 0.6727 - val_accuracy: 0.6406
Epoch 2/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5220 - accuracy: 0.7396 - val_loss: 0.4912 - val_accuracy: 0.7500
Epoch 3/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5044 - accuracy: 0.7639 - val_loss: 0.5014 - val_accuracy: 0.7656
Epoch 4/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5217 - accuracy: 0.7691 - val_loss: 0.5023 - val_accuracy: 0.7500
Epoch 5/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5495 - accuracy: 0.7309 - val_loss: 0.4989 - val_accuracy: 0.7422
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: 0.4867 - accuracy: 0.7760 - val_loss: 0.4956 - val_accuracy: 0.7383
Epoch 7/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5279 - accuracy: 0.7413 - val_loss: 0.5276 - val_accuracy: 0.7305
Epoch 8/100
1